## Recursive inference (multi-step) for BSCTRFM models

### use time series instead of SLDB arrays for easier and more efficient timestamp management

In [1]:
import os
import json
import numpy as np
import pandas as pd
import joblib
from datetime import datetime
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

# uncomment the following line for compatibility with TensorFlow 1.15 (on GCP)
# import tensorflow.compat.v1 as tf
# uncomment the following line for TensorFlow 2.X (local execution)
import tensorflow as tf

# forecast model was saved in TensorFlow 1.15
# but, in order to make predictions locally, has to be loaded with TensorFlow 2
from tensorflow.saved_model import load

In [2]:
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
# select a palette
from bokeh.palettes import d3
output_notebook()

Loading BokehJS ...

In [3]:
# symmetric mean absolute percentage error
def symmetric_mean_absolute_percentage_error(targets, predictions):
    '''
    predictions: a list with the predicted values
    targets: a list with the actual values
    '''
    import numpy as np
    # lists to NumPy arrays
    targets, predictions = np.array(targets), np.array(predictions)
    # verify predictions and targets have the same shape
    if predictions.shape == targets.shape:
            return(np.sum(2*np.abs(predictions - targets) /
                          (np.abs(targets) + np.abs(predictions)))/predictions.shape[0])

In [4]:
# a function to encode float values for serialized examples
def _float_feature_from_list_of_values(list_of_values):
    """Returns a float_list from a list of floats / doubles."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_values))

In [5]:
# converts a set of tensors to a feature dict to a serialized example to pass it
# to the prediction function of the saved model 
def input_tensors_to_serialized_example(encoder_input_float_tensor,
                                        decoder_input_float_tensor,
                                        id_float_tensor):
    # first, pass the float tensors to NumPy array, then flatten them
    encoder_input_flat_array = encoder_input_float_tensor.numpy().flatten()
    decoder_input_flat_array = decoder_input_float_tensor.numpy().flatten()
    id_flat_array = id_float_tensor.numpy().flatten()
    
    # second, build the protobuffer example
    example = tf.train.Example(
        # features within the example
        features=tf.train.Features(
            # feature definition
            feature={
                'encoder_input': _float_feature_from_list_of_values(encoder_input_flat_array),
                'decoder_input': _float_feature_from_list_of_values(decoder_input_flat_array),
                'id': _float_feature_from_list_of_values(id_flat_array)
            }
        )
    )    
    # third, serialize the example dictionary to a string
    serialized_example = example.SerializeToString()
    # fourth, wrap the serialized example as a NumPy-string array
    numpy_example = np.array(serialized_example, dtype='S')
    # fifth, wrap the NumPy-string array as a string tensor
    serialized_example = tf.convert_to_tensor(numpy_example)

    return serialized_example

In [6]:
PROJECT_ROOT = '/home/developer/gcp/cbidmltsf'

In [32]:
# during batch prediction, the model identifier is obtained via Abseil Flags
# remember this notebook is based on local execution,
# therefore model directory must be downloaded from GS before running the notebook
model_id = 'BSCTRFM_TPU_017'

In [33]:
# during batch prediction, the execution identifier is obtained via Abseil Flags
execution = 0

In [34]:
# during batch prediction, the SLDB identifier is obtained via Abseil Flags
# THE SLDB FOR INFERENCE MUST BE THE SAME USED FOR TRAINING! (THE ONE SETUP IN THE CONFIGURATION FILE)
sldb_id = 'LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168'

In [35]:
# during batch prediction, the dataset name is obtained via Abseil Flags
dataset = 'test'

In [36]:
# define a forecast window to guide the iterative prediction process
# start with a hourly, day-ahead process
forecast_window = 24

In [37]:
# ADD AN INFERENCE IDENTIFIER, BECAUSE FOR TRANSFORMER-BASED MODELS, DIFFERENT INFERENCES
# CAN BE PRODUCED FROM A SINGLE SAVED MODEL (USUALLY TO PRODUCE DIFFERENT FORECAST WINDOWS)
# during batch prediction, the inference identifier should be obtained via Abseil Flags
inference = '{:03d}'.format(forecast_window)

In [38]:
# build a path to the SLDB json file
data_dir = '{}/{}/{}'.format(PROJECT_ROOT, 'sldbs', sldb_id)

# then get the ts_identifier from the json file in the sldb directory
sldb_json_file = '{}/sldb.json'.format(data_dir)

In [39]:
# open the json file
with open(sldb_json_file, 'r') as inputfile:
    sldb_dict = json.load(inputfile)

In [40]:
# and get the time series identifier
ts_identifier = sldb_dict['ts']
ts_identifier

'LD2011-2014_SEPARATED_MT_320-MT_330'

In [41]:
# get the SLDB parameters for the forecasting model
config_json_file = '{}/{}/{}.json'.format(PROJECT_ROOT,
                                          'parameters',
                                          model_id)

# recover the sldb dictionary from the json file in parameters/
with open(config_json_file, 'r') as inputfile:
    configuration = json.load(inputfile)

In [42]:
m = sldb_dict['embedding']['hourly']
m

168

In [43]:
t = sldb_dict['no_targets']
t

168

In [44]:
# verify the values of the variables for batch inference
model_id, execution, dataset, inference

('BSCTRFM_TPU_017', 0, 'test', '024')

In [45]:
# build a list of saved models, given the parameters in the previous cell
saved_models = os.listdir('{}/models/{}_{:02d}/export/exporter'.format(PROJECT_ROOT, model_id, execution))

In [46]:
saved_models.sort()
saved_models

['1633532109', '1633532387', '1633532611', '1633532845', '1633533083']

In [47]:
encoder_input_columns = [
    'kw_scaled',
    'sin_hour_day',
    'cos_hour_day',
    'sin_day_week',
    'cos_day_week',
    'sin_day_month',
    'cos_day_month',
    'sin_day_year',
    'cos_day_year'
]

In [48]:
decoder_input_columns = encoder_input_columns

In [49]:
id_columns = ['token_id']

In [50]:
# now get the time series for the test dataset (unseen data)

In [51]:
# build the time series directory
time_series_folder = '{}/test'.format(data_dir)
time_series_folder

'/home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168/test'

In [52]:
! ls -l /home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_SEPARATED_MT_320-MT_330_BSCTRFM_168_168/test

total 572
-rw-rw-r-- 1 developer developer 49488 oct  3 10:40 MT_320.pkl
-rw-rw-r-- 1 developer developer 49488 oct  3 10:40 MT_321.pkl
-rw-rw-r-- 1 developer developer 49488 oct  3 10:40 MT_322.pkl
-rw-rw-r-- 1 developer developer 49488 oct  3 10:40 MT_323.pkl
-rw-rw-r-- 1 developer developer 49488 oct  3 10:40 MT_324.pkl
-rw-rw-r-- 1 developer developer 49488 oct  3 10:40 MT_325.pkl
-rw-rw-r-- 1 developer developer 49488 oct  3 10:40 MT_326.pkl
-rw-rw-r-- 1 developer developer 49488 oct  3 10:40 MT_327.pkl
-rw-rw-r-- 1 developer developer 49488 oct  3 10:40 MT_328.pkl
-rw-rw-r-- 1 developer developer 49488 oct  3 10:40 MT_329.pkl
-rw-rw-r-- 1 developer developer 49488 oct  3 10:40 MT_330.pkl


In [53]:
# use a dictionary to remain the code consistent with the SLDB building process
# most of the times, only ts['test'] will be used for inference
# however, ts['eval'] might also be used, as it have not really been seen by training process
# (no tranining modification resulted from evaluation stage)

### rename the time series dictionary to ts_test, use the customer_id as key

In [54]:
ts_test = dict()

In [55]:
# a dictionary to store predictions detail dataframe per customer id
predictions_detail = dict()

## skip the following code to avoid inference process

### set customer_id, load time series and scaler on training data

In [56]:
# a second inference identifier to run more than one inference
# on the same combination model_id, execution, inference;
# to produce different inference processes because predict_fn is stochastic
event = 0

for customer_id in ['MT_320', 'MT_321', 'MT_322', 'MT_323', 'MT_324', 'MT_325',
                    'MT_326', 'MT_327', 'MT_328', 'MT_329', 'MT_330']:
    
    # read the time series for the current customer
    ts_test[customer_id] = pd.read_pickle('{}/test/{}.pkl'.format(data_dir, customer_id))

    # for consistency, rename the column 'date' as 'timestamp'
    ts_test[customer_id] = ts_test[customer_id].rename(columns={"date": "timestamp"})
    # set the column 'timestamp' as index
    ts_test[customer_id] = ts_test[customer_id].set_index('timestamp')

    # report start and end timestamp for the loaded time series
    print('Loaded test time series for {}, which spans from {} to {}'.\
         format(customer_id,
                str(ts_test[customer_id].index[:1][0]),
                str(ts_test[customer_id].index[-1:][0])))

    # build a path to the scaler of the time series
    scaler_path = '{}/scalers/{}_min_max.save'.format(data_dir, customer_id)
    # and load it
    min_max_scaler = joblib.load(scaler_path)

    # pass the saved model identifier (it determines the number of training epochs)
    # and avoid using the latest saved model by default
    for saved_model_id in saved_models:

        # a columns list for the predictions dataframe
        pred_df_columns = ['model_id',
                           'execution',
                           'dataset',
                           'inference',
                           'customer_id',
                           'string_timestamps',
                           'predictions',
                           'targets']

        # build the predictions dataframe as a key-value pair of the dictionary
        predictions_detail[customer_id] = pd.DataFrame(columns=pred_df_columns)

        # use model identifier and execution number to build the model directory string
        model_dir = '{}_{:02d}'.format(model_id, execution)

        # get the path to the saved model main directory
        saved_model_path = '{}/{}/{}/export/exporter'.format(PROJECT_ROOT,
                                                             'models',
                                                             model_dir)

        # get all the files in the saved model path, to find the most recent one
        # all_files = os.listdir(saved_model_path)
        # get the path to the most recent saved model
        # latest_saved_model_id = sorted(all_files)[-1]

        # build the full path for the latest saved model dir
        export_dir = '{}/{}'.format(saved_model_path, saved_model_id)
        print ('Exported model path is {}'.format(export_dir))

        # load the saved model and the prediction function
        imported = load(export_dir=export_dir, tags='serve')
        predict_fn = imported.signatures["serving_default"]

        # iterate on a set of valid rows of the test dataset
        starting_point = 0 # based on the inference dataset
        span = 1 + 6*24 # number of days in the test dataset, expressed in hours
        dataset_row_indexes_list = starting_point + np.arange(span)

        for start_index in dataset_row_indexes_list:

            # define first prediction interval with start- and end-index
            # given the interval time_series[start_index:end_index]
            # the conditioning range is the union of the encoder-input and the decoder-input
            # and the prediction range is only the last lecture in the interval,
            # by means of a recursive inference process
            # on each step the last prediction is added to the decoder input
            # and the prediction range grows one step into the future

            # get the end-index of this recursive inference interval
            end_index = start_index + (m + t)

            # initialize a list to store recurrent predictions for this interval
            predictions_list = list()

            for i in np.arange(forecast_window):

                # build the inference interval as a sub-series of the dataset
                sub_series = ts_test[customer_id][start_index + i : end_index + i]

                # important: build sources as copies of the sub-series (and therefore of the global time series)
                # to avoid overwriting the original dataset

                # the encoder input source
                encoder_input = sub_series[encoder_input_columns][:m].copy()

                # the decoder input source
                # decoder_input = sub_series[m-1:-1].copy()
                decoder_input = sub_series[decoder_input_columns][m-1:m-1+t].copy()

                # the id (integer) for the customer
                id_input = sub_series[id_columns][:1].copy()

                # on first step (i=0), the decoder input carries only true values
                # and the predictions list is empty
                # on subsequent steps, the decoder input includes all previous predictions
                # (stored in the predictions list)
                if i > 0:
                    decoder_input['kw_scaled'][-i:] = predictions_list

                # the target source, for metrics calculation
                # the first part of the sub-series is the encoder input, and
                # the second part of the sub-series is the target (only the variable column!)
                target = sub_series['kw_scaled'][m:].copy()

                # build source tensors from the sub-series    
                encoder_input_tensor = tf.expand_dims(encoder_input, axis=0)
                decoder_input_tensor = tf.expand_dims(decoder_input, axis=0)
                id_tensor = tf.expand_dims(id_input, axis=0)

                # make input example for the prediction function
                input_example = input_tensors_to_serialized_example(encoder_input_tensor,
                                                                    decoder_input_tensor,
                                                                    id_tensor)

                # get the output of the prediction function as a dictionary
                predict_output_dict = predict_fn(input_example)

                # get the prediction output tensor
                predict_output_tensor = predict_output_dict['forecast']

                # get the most recent prediction
                most_recent_prediction = predict_output_tensor[0, :, 0].numpy()[-1]

                # append the most recent prediction timestep to the predictions list
                predictions_list.append(most_recent_prediction)

                # pass the predictions list to an array
                # current_predictions_array = np.array(predictions_list).reshape(-1, 1)
                # get the targets vector to be compared with the current predictions array
                # current_targets = np.array(target[-i-1:]).reshape(-1, 1)

                # calculate SMAPE on the rescaled variable
                # rescaled_predictions = min_max_scaler.inverse_transform(current_predictions_array)
                # rescaled_targets = min_max_scaler.inverse_transform(current_targets)

                # current_smape = symmetric_mean_absolute_percentage_error(
                #     rescaled_targets, rescaled_predictions)
                # print('On row {}, SMAPE for the first {} rescaled prediction(s) is {}'.format(start_index,
                #                                                                               i + 1,
                #                                                                               current_smape))        

            # iterative predictions over the forecast window reside in predictions_list
            # convert list to array, then expand feature dimension with value 1
            predicted_values = np.array(predictions_list).reshape(-1, 1)

            # inverse-scale predictions
            rescaled_predicted_values = min_max_scaler.inverse_transform(predicted_values)

            # and the true values remain in the prediction tensor, pass them to a NumPy array
            # for the true values array, expand feature dimension with value 1
            true_values = np.array(target[-i-1:]).reshape(-1, 1)

            # inverse-scale true values
            rescaled_true_values = min_max_scaler.inverse_transform(true_values)

            # a temporary dataframe built from the data in the current row
            df = pd.DataFrame(columns=pred_df_columns)
            df['model_id'] = [model_id]
            df['execution'] = [execution]
            df['dataset'] = [dataset]
            df['inference'] = [inference]
            df['customer_id'] = [customer_id]
            df['string_timestamps']= [pd.to_datetime(target.index[-i-1:]).astype(str).tolist()]
            df['predictions'] = [np.squeeze(rescaled_predicted_values).tolist()]
            df['targets'] = [np.squeeze(rescaled_true_values).tolist()]

            # calculate mean absolute error and normalized deviation
            mae = mean_absolute_error(rescaled_true_values, rescaled_predicted_values)
            df['mae'] = mae

            true_values_average = np.mean(rescaled_true_values)
            df['nd'] = mae/true_values_average

            # calculate root mean squared error and normalized root mean squared error
            rmse = sqrt(mean_squared_error(rescaled_true_values, rescaled_predicted_values))
            df['rmse'] = rmse
            df['nrmse'] = rmse/true_values_average

            df['smape'] = symmetric_mean_absolute_percentage_error(rescaled_true_values,
                                                                   rescaled_predicted_values)

            # append the temporary dataframe to the predictions detail dataframe
            predictions_detail[customer_id] = pd.concat([predictions_detail[customer_id], df])


        # reset the index of final dataframe, once all of its rows (dataset) have been processed
        predictions_detail[customer_id] = predictions_detail[customer_id].reset_index(drop=True)


        # hard-wired switcher to persist dataframe
        persist_detail = True
        
        if persist_detail:
            # build a path to persist the dataframe to database/predictions_detail/
            detail_pickle_path = '{}/{}/{}/{}_{:02d}_{}_{}_{}_{:02d}.pkl'.format(
                PROJECT_ROOT,
                'database',
                'predictions_detail',
                model_id,
                execution,
                saved_model_id,
                # for electricity dataset, replace dataset with customer_id
                customer_id,
                inference,
                event)

            # persist the Pandas dataframe to database/predictions_detail/
            predictions_detail[customer_id].to_pickle(detail_pickle_path)
            print(
                'Persisted predictions detail of {}_{:02d}_{}_{}_{}_{:02d}'.format(model_id,
                                                                                   execution,
                                                                                   saved_model_id,
                                                                                   customer_id,
                                                                                   inference,
                                                                                   event)
            )

Loaded test time series for MT_320, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_017_00/export/exporter/1633532109
Persisted predictions detail of BSCTRFM_TPU_017_00_1633532109_MT_320_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_017_00/export/exporter/1633532387
Persisted predictions detail of BSCTRFM_TPU_017_00_1633532387_MT_320_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_017_00/export/exporter/1633532611
Persisted predictions detail of BSCTRFM_TPU_017_00_1633532611_MT_320_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_017_00/export/exporter/1633532845
Persisted predictions detail of BSCTRFM_TPU_017_00_1633532845_MT_320_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_017_00/export/exporter/1633533083
Persisted predictions detail of BSCTRFM_TPU_017_00_1633533083_MT_320_0

Persisted predictions detail of BSCTRFM_TPU_017_00_1633532109_MT_328_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_017_00/export/exporter/1633532387
Persisted predictions detail of BSCTRFM_TPU_017_00_1633532387_MT_328_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_017_00/export/exporter/1633532611
Persisted predictions detail of BSCTRFM_TPU_017_00_1633532611_MT_328_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_017_00/export/exporter/1633532845
Persisted predictions detail of BSCTRFM_TPU_017_00_1633532845_MT_328_024_00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_017_00/export/exporter/1633533083
Persisted predictions detail of BSCTRFM_TPU_017_00_1633533083_MT_328_024_00
Loaded test time series for MT_329, which spans from 2014-08-18 01:00:00 to 2014-09-07 23:00:00
Exported model path is /home/developer/gcp/cbidmltsf/models/BSCTRFM_TPU_017_00/export/exporter/16335

## load predictions detail dataframes from the database directory

In [30]:
model_id = 'BSCTRFM_TPU_014'
execution = 4
saved_model_id = '1631533140'
inference = '024'

In [31]:
customer_ids = [
    'MT_320', 'MT_321', 'MT_322', 'MT_323', 'MT_324', 'MT_325', 'MT_326', 'MT_327', 'MT_328', 'MT_329', 'MT_330'
]

In [32]:
for customer_id in customer_ids:
    detail_pickle_path = '{}/{}/{}/{}_{:02d}_{}_{}_{}.pkl'.format(
        PROJECT_ROOT,
        'database',
        'predictions_detail',
        model_id,
        execution,
        saved_model_id,
        # for electricity dataset, replace dataset with customer_id
        customer_id,
        inference)
    
    predictions_detail[customer_id] = pd.read_pickle(detail_pickle_path)

In [33]:
# a dictionary to manage Bokeh plots
plots = dict()

In [34]:
for customer_id in customer_ids:
    # label the metrics plot per customer_id
    label = '{}_metrics'.format(customer_id)
    
    plots[label] = figure(
        title='ND, NRMSE, and SMAPE over the test set for {}'.format(customer_id),
        width=960,
        height=320,
        x_axis_label='Test Dataset Row Index',
        y_axis_label='Error')

    plots[label].line(
        x=predictions_detail[customer_id].index,
        y=predictions_detail[customer_id]['nd'],
        legend_label='ND',
        color='orange',
        line_width=2)

    plots[label].line(
        x=predictions_detail[customer_id].index,
        y=predictions_detail[customer_id]['nrmse'],
        legend_label='NRMSE',
        color='green',
        line_width=1)

    plots[label].line(
        x=predictions_detail[customer_id].index,
        y=predictions_detail[customer_id]['smape'],
        legend_label='SMAPE',
        color='purple',
        line_width=1)

    show(plots[label])

In [36]:
# now get a global, unique value for ND and NRSME for the 7 days in the test dataset

In [37]:
global_df = pd.DataFrame(columns=['customer_id', 'timestamp', 'prediction', 'target'])

In [38]:
# starting timestamps to predict over a complete day
start_row_ids = [0, 24, 48, 72, 96, 120, 144]

In [39]:
# iterate over customer_ids
for customer_id in customer_ids:
    
    # for each customer, iterate on starting rows for each day
    for start_row_id in start_row_ids:
        # make a buffer dataframe for one-day predictions
        buffer_df = pd.DataFrame()
        # populate the buffer dataframe

        # build a 24-time repeated list for the customer_id column
        buffer_df['customer_id'] = 24*[customer_id]
        # remember that predictions_detail dataframe stores lists,
        # then the index required to retrieve the list contents is the row index (find out why, later...)
        buffer_df['timestamp'] = pd.to_datetime(
            predictions_detail[customer_id]['string_timestamps'][start_row_id:start_row_id+1][start_row_id]
        )
        buffer_df['prediction'] = \
            predictions_detail[customer_id]['predictions'][start_row_id:start_row_id+1][start_row_id]

        buffer_df['target'] = \
            predictions_detail[customer_id]['targets'][start_row_id:start_row_id+1][start_row_id]

        # buffer_df = buffer_df.set_index('timestamp')

        global_df = pd.concat([global_df, buffer_df])

In [40]:
global_df

,customer_id,timestamp,prediction,target
0,MT_320,2014-09-01 00:00:00,69.997803,74.781215
1,MT_320,2014-09-01 01:00:00,69.530457,69.396600
2,MT_320,2014-09-01 02:00:00,66.366066,63.776477
3,MT_320,2014-09-01 03:00:00,62.189232,64.335284
4,MT_320,2014-09-01 04:00:00,55.775852,61.266722
...,...,...,...,...
19,MT_330,2014-09-07 19:00:00,129.202454,141.406250
20,MT_330,2014-09-07 20:00:00,121.608742,130.750000
21,MT_330,2014-09-07 21:00:00,117.416702,134.042188
22,MT_330,2014-09-07 22:00:00,93.051590,100.467187


### over the 7 days of the test dataset, daily forecasting windows, starting at midnight (2014-09-0X 00:00:00)

In [41]:
# get a global value for MAE
global_mae = mean_absolute_error(global_df['prediction'], global_df['target'])
global_mae

38.7537443218111

In [42]:
# get a global average for true values
global_true_values_average = np.mean(global_df['target'])
global_true_values_average

498.16469337378595

In [43]:
# get a global value for ND
global_nd = global_mae/global_true_values_average
global_nd

0.07779303679543013

In [44]:
# get a global value for RMSE
global_rmse = sqrt(mean_squared_error(global_df['prediction'], global_df['target']))
global_rmse

80.13435767330789

In [45]:
# get a global value for NRMSE
global_nrmse = global_rmse/global_true_values_average
global_nrmse

0.16085916713728443

In [46]:
# get a global vale for SMAPE
global_smape = symmetric_mean_absolute_percentage_error(global_df['prediction'], global_df['target'])
global_smape

0.07562048439581362

In [47]:
for customer_id in customer_ids:
    
    label = '{}_predictions'.format(customer_id)

    plots[label] = figure(
        title='Predictions over the test dataset for {}'.format(customer_id),
        width=960,
        height=320,
        x_axis_type='datetime',
        x_axis_label='Timestmap',
        y_axis_label='Value')

    plots[label].line(
        x=global_df[global_df['customer_id'] == customer_id]['timestamp'],
        y=global_df[global_df['customer_id'] == customer_id]['prediction'],
        legend_label='predictions',
        color='red',
        line_width=1)

    plots[label].line(
        x=global_df[global_df['customer_id'] == customer_id]['timestamp'],
        y=global_df[global_df['customer_id'] == customer_id]['target'],
        legend_label='targets',
        color='blue',
        line_width=1)

    show(plots[label])